### - model validation on pre-pared data
### - make submit

In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import dill
import sys
import pandas as pd
import numpy as np
import gc
import warnings
  
from dstools.spark import init_spark2

from dstools.ml import yandex_mean_encoder

from sklearn.model_selection import train_test_split

from sklift.models import SoloModel, ClassTransformation, TwoModels
from sklift.preprocess import balancer
from sklift.metrics import uplift_at_k

from causalml.inference.tree import UpliftTreeClassifier, UpliftRandomForestClassifier
from causalml.inference.tree import uplift_tree_string, uplift_tree_plot 

from sklearn.pipeline import Pipeline

from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

warnings.filterwarnings('ignore') 

In [3]:
spark = init_spark2({
    "appName": "ret_baseline",
    "spark.yarn.queue": "default",
    "spark.driver.memory": "32g",
    "spark.executor.memoryOverhead": "2048",
    "spark.driver.extraJavaOptions": "-XX:ThreadStackSize=81920",
    "spark.executor.extraJavaOptions": "-XX:ThreadStackSize=81920",
    "spark.driver.maxResultSize": "45g",
    "spark.driver.memory": "45g"
})

In [4]:
sys.path.append('./scripts/')
from add_functions import catb_get_feature_imp, make_validation

-----

### load data

In [5]:
# read all data
sdf_cliens = spark.read.csv('/user/mvshevc5/retailhero/clients.csv', inferSchema=True, header=True)
sdf_products = spark.read.csv('/user/mvshevc5/retailhero/products.csv', inferSchema=True, header=True)
sdf_purchase = spark.read.csv('/user/mvshevc5/retailhero/purchases.csv', inferSchema=True, header=True)
sdf_train = spark.read.csv('/user/mvshevc5/retailhero/uplift_train.csv', inferSchema=True, header=True)
sdf_test = spark.read.csv('/user/mvshevc5/retailhero/uplift_test.csv', inferSchema=True, header=True)

In [8]:
df_clients = sdf_cliens.toPandas()
df_train = sdf_train.toPandas()
df_test = sdf_test.toPandas()

df_clients = df_clients.set_index('client_id')
df_train = df_train.set_index('client_id')
df_test = df_test.set_index('client_id')

In [21]:
# baseline features data:
base_train = pd.read_csv('data/feats_baseline_train.csv', index_col=0)
base_train = base_train.drop(['treatment_flg','target'], axis=1)
base_train.columns  = ['base_' + col for col in base_train.columns.tolist()]

print(base_train.shape)
base_train.head(2)

(200039, 23)


,base_first_issue_date,base_first_redeem_date,base_age,base_gender,base_total_trans_count,base_last_month_trans_count,base_regular_points_received_sum_all,base_express_points_received_sum_all,base_regular_points_spent_sum_all,base_express_points_spent_sum_all,base_purchase_sum_sum_all,base_store_id_sum_all,base_regular_points_received_sum_last_month,base_express_points_received_sum_last_month,base_regular_points_spent_sum_last_month,base_express_points_spent_sum_last_month,base_purchase_sum_sum_last_month,base_store_id_sum_last_month,base_first_issue_date_weekday,base_first_redeem_date_weekday,base_first_issue_date_hour,base_first_redeem_date_hour,base_diff
000012768d,1.501948e+09,1.515094e+09,45,85,4,2,25.7,0.0,0.0,0.0,2803.0,3,10.0,0.0,0.0,0.0,1222.0,1,5,3.0,15,19.0,13146559.0
000036f903,1.491832e+09,1.492951e+09,72,70,32,8,54.9,60.0,0.0,0.0,9805.0,5,13.7,0.0,0.0,0.0,2784.0,4,0,6.0,13,12.0,1118613.0


In [22]:
base_test = pd.read_csv('data/feats_baseline_test.csv', index_col=0)
base_test = base_test.drop(['target'], axis=1)
base_test.columns  = ['base_' + col for col in base_test.columns.tolist()]

print(base_test.shape)
base_test.head(2)

(200123, 23)


,base_first_issue_date,base_first_redeem_date,base_age,base_gender,base_total_trans_count,base_last_month_trans_count,base_regular_points_received_sum_all,base_express_points_received_sum_all,base_regular_points_spent_sum_all,base_express_points_spent_sum_all,base_purchase_sum_sum_all,base_store_id_sum_all,base_regular_points_received_sum_last_month,base_express_points_received_sum_last_month,base_regular_points_spent_sum_last_month,base_express_points_spent_sum_last_month,base_purchase_sum_sum_last_month,base_store_id_sum_last_month,base_first_issue_date_weekday,base_first_redeem_date_weekday,base_first_issue_date_hour,base_first_redeem_date_hour,base_diff
000048b7a6,1.544881e+09,-9.223372e+09,68,70,8,1,26.5,0.0,0.0,0.0,3772.0,2,1.2,0.0,0.0,0.0,342.0,1,5,NaN,13,NaN,-1.076825e+10
000073194a,1.495544e+09,1.511522e+09,60,70,17,6,74.9,0.0,-96.0,0.0,9601.4,1,25.0,0.0,0.0,0.0,3393.3,1,1,4.0,12,11.0,1.597811e+07


In [23]:
# transform clients data

df_clients['first_issue_date_weekday'] = df_clients['first_issue_date'].dt.weekday
df_clients['first_redeem_date_weekday'] = df_clients['first_redeem_date'].dt.weekday
df_clients['first_issue_date_hour'] = df_clients['first_issue_date'].dt.hour
df_clients['first_redeem_date_hour'] = df_clients['first_redeem_date'].dt.hour
df_clients['first_issue_date'] = df_clients['first_issue_date'].astype(int)/10**9
df_clients['first_redeem_date'] = df_clients['first_redeem_date'].astype(int)/10**9
df_clients['diff'] = df_clients['first_redeem_date']-df_clients['first_issue_date']
df_clients['gender'] = list(ord(v[0]) for v in df_clients['gender'].values)

df_clients.columns = ['cl_' + col for col in df_clients.columns.tolist()]

In [24]:
# load all trans data:
df_total = pd.read_csv('data/putch_total_agg.csv', index_col=0)
df_total_cat = pd.read_csv('data/putch_total_agg_cat.csv', index_col=0)
df_total_date = pd.read_csv('data/putch_total_agg_date.csv', index_col=0)
df_last = pd.read_csv('data/putch_last_agg.csv', index_col=0)

In [35]:
df_full_train_feats = pd.concat([
    df_clients, df_total, df_total_cat, df_total_date, df_last, base_train
], axis=1)

df_full_train = df_full_train_feats.join(df_train, how='right')

df_full_test_feats = pd.concat([
    df_clients, df_total, df_total_cat, df_total_date, df_last, base_test
], axis=1)

df_full_test = df_full_test_feats.join(df_test, how='right')

del df_full_train_feats, df_full_test_feats
gc.collect()

print(f'Train shape {df_full_train.shape}')
print(f'Test shape {df_full_test.shape}')

#Train shape (200039, 129)
#Test shape (200123, 127)
df_full_train.head(2)

Train shape (200039, 129)
Test shape (200123, 127)


,cl_first_issue_date,cl_first_redeem_date,cl_age,cl_gender,cl_first_issue_date_weekday,cl_first_redeem_date_weekday,cl_first_issue_date_hour,cl_first_redeem_date_hour,cl_diff,total_count,transaction_id__nunique,regular_points_received__mean,regular_points_received__max,regular_points_received__min,regular_points_received__median,express_points_received__mean,express_points_received__max,express_points_received__min,express_points_received__median,regular_points_spent__mean,regular_points_spent__max,regular_points_spent__min,regular_points_spent__median,express_points_spent__mean,express_points_spent__max,express_points_spent__min,express_points_spent__median,purchase_sum__mean,purchase_sum__max,purchase_sum__min,purchase_sum__median,product_quantity__mean,product_quantity__max,product_quantity__min,product_quantity__median,trn_sum_from_iss__mean,trn_sum_from_iss__max,trn_sum_from_iss__min,trn_sum_from_iss__median,trn_sum_from_red__mean,trn_sum_from_red__max,trn_sum_from_red__min,trn_sum_from_red__median,netto__mean,netto__max,netto__min,netto__median,most_comm_product_id,nunique_product_id,most_comm_cnt_ratio_product_id,most_comm_sum_ratio_product_id,most_comm_transaction_id,nunique_transaction_id,most_comm_cnt_ratio_transaction_id,most_comm_sum_ratio_transaction_id,most_comm_store_id,nunique_store_id,most_comm_cnt_ratio_store_id,most_comm_sum_ratio_store_id,most_comm_level_1,nunique_level_1,most_comm_cnt_ratio_level_1,most_comm_sum_ratio_level_1,most_comm_level_2,nunique_level_2,most_comm_cnt_ratio_level_2,most_comm_sum_ratio_level_2,most_comm_level_3,nunique_level_3,most_comm_cnt_ratio_level_3,most_comm_sum_ratio_level_3,most_comm_level_4,nunique_level_4,most_comm_cnt_ratio_level_4,most_comm_sum_ratio_level_4,most_comm_segment_id,nunique_segment_id,most_comm_cnt_ratio_segment_id,most_comm_sum_ratio_segment_id,most_comm_vendor_id,nunique_vendor_id,most_comm_cnt_ratio_vendor_id,most_comm_sum_ratio_vendor_id,min_date_trans,max_date_trans,days_from_last_trans,lats_total_count,lats_transaction_id__nunique,lats_regular_points_received__mean,lats_regular_points_received__max,lats_regular_points_received__min,lats_regular_points_received__median,lats_regular_points_spent__mean,lats_regular_points_spent__max,lats_regular_points_spent__min,lats_regular_points_spent__median,lats_purchase_sum__mean,lats_purchase_sum__max,lats_purchase_sum__min,lats_purchase_sum__median,lats_product_quantity__mean,lats_product_quantity__max,lats_product_quantity__min,lats_product_quantity__median,base_first_issue_date,base_first_redeem_date,base_age,base_gender,base_total_trans_count,base_last_month_trans_count,base_regular_points_received_sum_all,base_express_points_received_sum_all,base_regular_points_spent_sum_all,base_express_points_spent_sum_all,base_purchase_sum_sum_all,base_store_id_sum_all,base_regular_points_received_sum_last_month,base_express_points_received_sum_last_month,base_regular_points_spent_sum_last_month,base_express_points_spent_sum_last_month,base_purchase_sum_sum_last_month,base_store_id_sum_last_month,base_first_issue_date_weekday,base_first_redeem_date_weekday,base_first_issue_date_hour,base_first_redeem_date_hour,base_diff,treatment_flg,target
client_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
000012768d,1.501948e+09,1.515094e+09,45,85,5,3.0,15,19.0,13146559.0,52,4,7.551923,10.0,2.0,8.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,784.788462,1007.0,419.0,803.0,1.038462,4.0,0.0,1.0,53.942308,170.0,0.0,50.0,NaN,NaN,NaN,NaN,0.540231,1.5,0.018,0.40,057ea8df98,46,0.038462,0.024333,7e3e2e3984,4,0.365385,0.468843,017c89b915,3,0.423077,0.376437,e344ab2e71,3,0.519231,0.545002,ad2b2e17d2,8,0.326923,0.312627,ca69ed9de2,24,0.230769,0.215467,46951c62e8,31,0.076923,0.069298,1.0,23.0,0.078431,0.073518,43acd80c1a,29,0.076923,0.083094,1.543648e+09,1.552576e+09,292,22,2,6.363636,8.0,2.0,8.0,0.0,0.0,0.0,0.0,698.272727,803.0,419.0,803.0,0.863636,2.0,0.0,

-----

#### MODEL

In [46]:
try_feat = list(set(df_full_train.columns.tolist()) - set(['target', 'treatment_col']))

cat_cols = df_full_train.select_dtypes(include='object').columns.tolist()
cat_cols = list(set(cat_cols).intersection(set(try_feat)))

In [77]:
with open('models/uplift_my_and_baseline_feats_two_models_catb.dill', 'rb') as f_in:
    model = dill.load(f_in)
    
feat_imp = catb_get_feature_imp(
    catb_est=model.estimator_trmnt,
    features_list=df_full_train\
        .drop(['treatment_flg', 'target'], axis=1)\
        .columns\
        .tolist(), 
    n_top=80
)

top_feats = feat_imp['feat'].values.tolist()

cat_top_feats = df_full_train[top_feats]\
    .select_dtypes(include='object')\
    .columns\
    .tolist()
    
try_feat = list(
   set(top_feats) - set(cat_top_feats)
)

len(try_feat)

74

In [108]:
## PREPARE DATA FOR XGB / LGBM MODEL, fit transform target_encodet on train data
enc = yandex_mean_encoder(
    columns=cat_top_feats, alpha=100
)

xgb_est_params = {
    'max_depth':2, 
    'learning_rate':0.5, 
    'n_estimators':20,
    'nthread':30,
    'n_gpus':0,
    'seed':234
}

estimator = XGBClassifier(**xgb_est_params)

estimator_trmnt = XGBClassifier(**xgb_est_params)
estimator_ctrl = XGBClassifier(**xgb_est_params)

uplift_model_2mod = TwoModels(
    estimator_trmnt=estimator_trmnt, 
    estimator_ctrl=estimator_ctrl
)

uplift_model_cl_tr = ClassTransformation(
    estimator=estimator
)

upl_xgb_cl_tr_pipeline = Pipeline(steps=[
    ('enc', enc),
    ('est', uplift_model_cl_tr)
])

###### SAME FOR 2 mdodels

In [ ]:
try_feat = list(set(try_feat) - set(cat_cols))

val_res_all_feats_xgb_2mod = make_validation(
    df_full=df_full_train,
    upift_model=uplift_model_2mod,
    try_feat=try_feat,
    n_iter=20
)

val_res_all_feats_xgb_cl_tr = make_validation(
    df_full=df_full_train,
    upift_model=uplift_model_cl_tr,
    try_feat=try_feat,
    n_iter=20
)

In [ ]:
try_feat = top_feats

val_res_all_feats_pipeline = make_validation(
    df_full=df_full_train,
    upift_model=upl_xgb_cl_tr_pipeline,
    try_feat=try_feat,
    pipeline_flg=True,
    n_iter=20
)

In [107]:
val_res_all_feats_xgb_cl_tr_df = pd.DataFrame(val_res_all_feats_xgb_cl_tr).T
val_res_all_feats_pipeline_df = pd.DataFrame(val_res_all_feats_pipeline).T

print(
    np.round(val_res_all_feats_xgb_cl_tr_df['score_val'].mean(), 5), 
    np.round(val_res_all_feats_xgb_cl_tr_df['score_val'].std(), 5)
)

print(
    np.round(val_res_all_feats_xgb_2mod_df['score_val'].mean(), 5), 
    np.round(val_res_all_feats_xgb_2mod_df['score_val'].std(), 5)
)

print(
    np.round(val_res_all_feats_pipeline_df['score_val'].mean(), 5), 
    np.round(val_res_all_feats_pipeline_df['score_val'].std(), 5)
)

# ALL cont 127 feats: 
# 0.55418 / 0.00949

# TOP 76 cont feats:
# 0.07135 / 0.00708

# TOP 76 cont feats + cat top 6 feats:
# 0.07233 0.00512 
# 0.07038 0.00508

0.0699 0.00691
0.0699 0.00691
0.07038 0.00508


-----

##### Learn on full train df and submit:

In [ ]:
uplift_model_cl_tr.fit(
    X=df_full_train[try_feat],
    y=df_full_train['target'],
    treatment=df_full_train['treatment_flg']   
)

In [65]:
upl_final_pred_test = uplift_model_cl_tr.predict(df_full_test[try_feat])

In [66]:
df_submit = df_full_test.assign(uplift=upl_final_pred_test)[['uplift']]

print(df_submit.shape)
df_submit.head(2)

(200123, 1)


,uplift
client_id,
000048b7a6,0.046865
000073194a,0.057391


In [67]:
df_submit.to_csv('submissions/sub8_myrealfeats_xgb_d2_cl_tr.csv')
!head -2 'submissions/sub8_myrealfeats_xgb_d2_cl_tr.csv'

# Public: 0,0874

client_id,uplift
000048b7a6,0.04686451


----